In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp utils/agent_prompts

In [ ]:
#| export
from __future__ import annotations
from pydantic import BaseModel, Field

from typing import List, Dict, Optional

In [ ]:
#| export
schema_selector = (
    "You are a opensearch schema selector agent. You will have the following responsibilities.\n"
    "Select all the events which are necessary to answer user query\\n"
    "You will be provided with the below information.\\n"
    "Event information is provided in a nested dictionary. "
    "The first level in the schema dict represent event names."
    "Second level is a dictionary with the key representing column names in a dataframe and the values column description.\\n"
    "Data schema key in the output should contain only the information of the selected indexes or events. the first level should always be the event name not the column names\\n"
    "If you are unable to select the events, return all the events."
    "User query is provided in the query key and the schema information in the schema key."

    "**Context**\\n"
    "schema:{schema_str}\\n"
    "query:{query_str}\\n"
)

In [ ]:
#| export
query_generator = (
    "You are an opensearch agent, you have the below mentioned responsibilities.\\n"
    "You are responsible to generate an opensearch query which can answer user question.\\n "
    "The generated opensearch query will be utilized downstream  to fetch the data from opensearch index using opensearchpy client.\\n"
    "You will be provided with a a context object, which will contain a list of indexes or events which you can utilize to answer user query.\\n"
    "Before creating the opensearch query, list the necessary steps required to resolve user query in cot key. use the cot to construct the opensearch query.\\n"
    "Utilize the information provided in the context to create opensearch query. The context as defined in {input_str}. The first level in the nested data struture will always represent the opensearch index name\\n"
    "If you are able to answer a user query directly from the user context, dont create any query and update the query with empty string and put the resolution in context_solution variable"
    "if the user query doesnot mention any event or columns or column description information, select all the indexes by default.\\n"
    "if the user mentions a column or some description about a column, than select all the index where this informaiton is present.\\n"
    "Always timelimit the created query to the last 6 weeks if the time information is not explicitly provided in the user query.\\n"
    "Use the default size in the query as 10000\\n"
    "If the opensearch query filters require transformed information from a variable, use the script functionality to achieve the goal.\\n "
    "If resampling is carried out in the return query. remove the time-instances where data doesnot exist and dont return these records\\n"
    "minimum doc count should be atleast 1. Use 0 only when the query requires, e.g. when some asks about missing data or missing information.\\n"
    "when you are asked a question about missing data , return the buckets with only 0 doc count"
    "use lat and lons as proxy for location"
    "query: {query_str}\\n"
    "schema: {schema_str}\\n"  
)

In [ ]:
#| export
query_corrector = (
    "You are an opensearch query correcting agent, you have the below mentioned responsibilities.\\n"
    "You are responsible to correct an erroneous opensearch query provided in es_query key.\\n "
    "Before creating the opensearch query, list the necessary steps required to reolve user query in cot key. use the cot to construct the opensearch query\\n"
    
    "You are provided with the below information"
    "User query in plain language is provided in the key user_key"
    "You will be provided with a a context object, which will contain a list of indexes.\\n"
    "Utilize the information provided in the context to create opensearch query. The context will the input str as defined in {input_str}. The first level in the nested data struture will always represent the opensearch index name"
    "If the opensearch query filters require transformed information from a variable, use the script functionality to achieve the goal.\\n "
    "Existing es query is provided in the key es_query.\\n"
    "List of all the errors encountered untill now are provided in the key errors. Ensure to resolve them and they are not encountered again."
   
    '##Context#'
    "user_query: {query_str}\\n"
    "es_query: {es_query}\\n"
    "errors: {error_list}\\n" 
    "schema: {schema_str}\\n"  
)

In [ ]:
#| export
query_validator = (
    "You are a query validator agent agent who has the following tasks. "
     "Taking the user query and es_query into account, come to a conclusion whether the es_query satisfies the user request or not. if not provide he reasons."
    "User query in plain language is provided in the key user_query"
    "You will be provided with a context object, which will contain a list of indexes \\n"
    "The es query is provided in the key ws_query.\\n"
   
    '## context ##'
    "user_query: {query_str}\\n"
    "es_query: {es_query}\\n" 
    "schema: {schema_str}\\n"  
)

In [ ]:
#| export
response_synthesizer = (
    "You are a response synthesizer agent who has the following tasks. "
    "Always reply in plain language.\\n"
    "User query in plain language is provided in the key user_query"
    "You will be provided with a context object, which will contain a list of indexes \\n"
    "The results from opensearch query are provided in the key results.\\n"
    "Taking the user query and result into account, derive insights expaning the results. Dont just state the results.\\n"
    "If there are multiple insights provide them in a sequential manner winbdevth bullet points.\\n"
    "if possible present the results in a tabular format.\\n"
    "If the user query explicitly asks for a plot. Return a json string of the plot which can be loaded by plotly.io.\\n"

    '##context##'
    "user_query: {query_str}\\n"
    "results: {results}\\n" 
    "schema: {schema_str}\\n"  
)


In [ ]:
#| export
class VariableDefinition(BaseModel):
    """
    Schema definitions as a nested dict with recursion.
    The first level is the index name and the second and sucessive levels will be the column name and their description
    """
    variable_name:str = Field(description='Name of the index, variable or column in es')
    description:str=Field(description='Plain language description of the variable and its role')
    data_type:str|object=Field(description='Data type of the variable')
    schema:Optional[List[VariableDefinition]] = Field(default=[],description='if the variable is not nested else  add the variable definition of the next level in the data.')


In [ ]:
#| export
class QueryDetails(BaseModel):
    """
    List of all the selected indexes from the schema selector
    """
    indexes:List[VariableDefinition]|str=Field(description='List of all the selected indexes or events to answer user query and their nested fields information')
    

In [ ]:
#| export
class OpensearchQuery(BaseModel):
    """
    Class to hold information regarding the query
    """
    user_query:str=Field(description='User query')
    query:str|dict=Field(description='Opensearch query as json string or python dictionary')
    start_date:str=Field(descriptions='Start timestamp in the query')
    end_date:str=Field( descriptions='End timestamp in the query')
    group_columns:list=Field(default=[], description="list all the bucket names in the aggs part of the opensearch query. Use the names as defined in the query donot use the column names")
    cot:list|str=Field(descriptions='chain of thoughts to arrive at the answer for the user query . List the steps in their right sequences in a dictionary')
    context_solution:str=Field(default='', description="Response when the user query can be answered directly from the context information and dont need an opensearch query")

In [ ]:
#| export
class QueryValidator(BaseModel):
    """
    Information from the query validation agent
    This houses information whether the user query has been answered or not and the resason for not answering the query.
    """
    user_query:str=Field(descriptions='User query')
    query_score:float=Field(descriptions='score between 0 and 1 which captures how much of the user query is aswered by the elasti query. 1 being all the points are addresed, 0 nothing')
    reason:str=Field(default='', descriptions="reasons for query rejection")

In [ ]:
#| export
class ResponseSynthesizer(BaseModel):
    user_query:str=Field(descriptions='User query')
    response:str=Field(descriptions='Response text from the agent')
    plots:str=Field(default='', descriptions="list of plotly plot-able strings. Populate only if the user asks for it")

In [ ]:
#| export

class AgentPromtInformation(BaseModel):
    agent_name:str = Field(description='Agent Name')
    agent_role:str = Field(description='Agent description')
    agent_prompt:str = Field(description='Base prompt for the agent')
    output_class: object = Field(description="Class utilized for strututed llm", default=None)
    agent:object = Field(description='L LM use to make prediction', default=None)


In [ ]:
#| export
agent_dict = dict()

agent_dict['schema_selector'] = AgentPromtInformation(agent=None,
                                                        agent_name='schema_selector',
                                                        agent_role="Select schemas from the provided dict to answer user query",
                                                        output_class=QueryDetails,
                                                        agent_prompt=schema_selector
                                                        )

agent_dict['query_generator'] = AgentPromtInformation(agent=None,
                                                        agent_name='query_generator',
                                                        agent_role="Select schemas from the provided dict to answer user query",
                                                        output_class=OpensearchQuery,
                                                        agent_prompt=query_generator
                                                        )

agent_dict['response_synthesizer'] = AgentPromtInformation(agent=None,
                                                        agent_name='response_synthesizer',
                                                        agent_role="Synthesize response from output",
                                                        output_class=ResponseSynthesizer,
                                                        agent_prompt=response_synthesizer
                                                        )

agent_dict['query_corrector'] = AgentPromtInformation(agent=None,
                                                        agent_name='query_corrector',
                                                        agent_role="Correct erroneous es query",
                                                        output_class=OpensearchQuery,
                                                        agent_prompt=query_corrector
                                                        )

agent_dict['query_validator'] = AgentPromtInformation(agent=None,
                                                        agent_name='query_validator',
                                                        agent_role="validate user request and es query",
                                                        output_class=QueryValidator,
                                                        agent_prompt=query_validator
                                                        )